In [1]:
import os
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hashlib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from six.moves import urllib
Download_Root = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
House_Path = "datasets/housing"
House_Url = Download_Root + House_Path + '/housing.tgz'

'''获取数据'''
def fetch_housing_data(housing_path=House_Path, housing_url=House_Url):  
    if(os.path.isdir(housing_path)==False):    
        os.makedirs(housing_path)  #创建目录
    tgz_path = os.path.join(housing_path, 'housing.tgz')  #合并数据
    if(os.path.exists(tgz_path)==False):
        urllib.request.urlretrieve(housing_url, tgz_path)  #网上下载housing.tgz数据包
        housing_tgz = tarfile.open(tgz_path)  #打开压缩未见
        housing_tgz.extractall(housing_path)  #将压缩文件进行提取
        housing_tgz.close()    #提取完成后关闭
'''加载csv格式的房屋数据'''
def load_housing_data(housing_path=House_Path):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

'''hash采样'''
'''利用hash序列选取数据达到每次选取的测试集不变'''
'''这里以index作为id，做好用数据中不变的特征经过适当处理作为id，例如经纬度乘以10'''
def test_set_check(identifier, test_ratio, func):
    return func(np.int64(identifier)).digest()[-1] < 256 * test_ratio
    #这里若不使用匿名函数，hashlib只返回index列第一个数的结果
def split_train_test_by_id(data, test_ratio, id_column, func=hashlib.md5):
    ids = data[id_column]
    #in_test_set 为布尔量
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, func)) 
    return data.loc[~in_test_set], data.loc[in_test_set]
#housing_with_id = df.reset_index()  #添加了index这一column作为数据的id
#train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

'''sklearn调用函数进行采样'''
#train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

'''分层采样'''
'''例如需要在城市中抽1000人做调查，不可以随机抽取'''
'''若该城市男女比例为6:4,则应该选600男400女，这样的采样数据更能代表整个数据集'''
#df['income_cat'] = np.ceil(df.median_income / 1.5) #创建新列，缩小数据值便于分组
##不满足条件的值用5替代，inplace=True表示原地替换
#df['income_cat'].where(df.income_cat < 5, other=5, inplace=True) 
##分层采样函数 n_splits表示进行洗牌的次数，可以洗三次，取第三次分层结果
#split_data = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
#for train_index,test_index in split_data.split(df, df.income_cat):  #输入x和y
#    strat_train_set = df.loc[train_index]
#    strat_test_set = df.loc[test_index]

#1获取网络上的数据
#2加载数据至程序中
#3采样测试集(hash采样，sklearn调用函数采样，分层采样）
#4数据探索，可视化，发现规律
#5为机器学习算法准备数据
#6选择并训练模型
#7模型微调
#8分析最佳模型和他们的误差
#1
fetch_housing_data()    
#2
df = load_housing_data() 
#df.info()  #返回df的信息
#df['ocean_proximity'].value_counts()  #该columns的值为类别，可以用value_counts查看有那些类别和类别的数量
#df.describe()  #输出每个column的mean,std, max, min等参数
#df.hist(bins=50, figsize=(20, 15))  #生成柱状图
#plt.show()
#3
df['income_cat'] = np.ceil(df.median_income / 1.5) #创建新列，缩小数据值便于分组 ceil向上取整
#不满足条件的值用5替代，inplace=True表示原地替换
df['income_cat'].where(df.income_cat < 5, other=5, inplace=True) 
split_data = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index,test_index in split_data.split(df, df.income_cat):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]
for values in [strat_train_set, strat_test_set]:
    values.drop('income_cat', axis=1, inplace=True)
#4
'''如果训练集很大，可以采样一个探测集用以发现数据的规律(探索）'''
rep_train = strat_train_set.copy() #创建一个副本，以免损伤训练集
#可视化
#类型为散点图 x为rep_train中经度值 y为纬度值 透明度0.1 形状以population设置 
#图例为population 颜色以房屋价格中位数设置 cmap为jet型 colorbar可以改为False就知道是干嘛的了
plt.figure()
rep_train.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1, s=rep_train.population/100,
               label='population', c='median_house_value', cmap=plt.cm.jet, colorbar=True)
plt.show()
# 发现规律
#相关系数的范围是-1到1。当接近1时,意味强正相关;当相关系数接近-1时,意味强负相关;相关系数接近0,意味没有线性相关性
#相关系数只测量线性关系，可能完全忽视非线性关系
corr_matrix = rep_train.corr()  # 输出所有特征之间的相关系数矩阵
corr_matrix.median_house_value.sort_values(ascending=False)  # 对房价中位数与其他特征的相关系数进行排序输出
# pd中有scatter_matrix函数能画出每个数值属性对每个其它数值属性的图
# from pandas.plotting import scatter_matrix
# attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
# scatter_matrix(rep_train[attributes], figsize=(12, 8))
# plt.show()
'''最重要的是收入中位数与房价中位数的图，这张图说明了几点。首先,相关性非常高;
可以清晰地看到向上的趋势,并且数据点不是非常分散。第二,我们之前看到的最高价,
清晰地呈现为一条位于 500000 美元的水平线。这张图也呈现了一些不是那么明显的
直线:一条位于 450000 美元的直线,一条位于 350000 美元的直线,一条在 280000 美元
的线,和一些更靠下的线,这些数据的巧合需要在给算法提供数据之前将其去除'''
# rep_train.plot(kind="scatter", x="median_income",y="median_house_value", alpha=0.5)
# plt.show()

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 2 Axes>

'最重要的是收入中位数与房价中位数的图，这张图说明了几点。首先,相关性非常高;\n可以清晰地看到向上的趋势,并且数据点不是非常分散。第二,我们之前看到的最高价,\n清晰地呈现为一条位于 500000 美元的水平线。这张图也呈现了一些不是那么明显的\n直线:一条位于 450000 美元的直线,一条位于 350000 美元的直线,一条在 280000 美元\n的线,和一些更靠下的线,这些数据的巧合需要在给算法提供数据之前将其去除'

In [2]:
#属性组合测试
rep_train['rooms_per_household'] = rep_train['total_rooms'] / rep_train['households']# 每户家庭拥有的房间数
rep_train['bedrooms_pre_room'] = rep_train['total_bedrooms'] / rep_train['total_rooms']# 卧室数目占总房间数目
rep_train['population_per_household'] = rep_train['population'] / rep_train['households']# 每户的人口数
corr_matrix = rep_train.corr() #相关系数矩阵，正相关以及负相关都具有信息
corr_matrix['median_house_value'].sort_values(ascending=False) #排序，降序
'''这一步的数据探索不必非常完备,此处的目的是有一个正确的开始,快速发现规律,以得到
一个合理的原型(可以想象成函数，哪个是x，函数系数又是多少)。但是这是一个交互过程:一旦你得到了一个原型,并运行起来,你就可以
分析它的输出,进而发现更多的规律,然后再回到数据探索这步。'''
#5
#为机器学习算法准备数据
rep_train = strat_train_set.drop('median_house_value', axis=1)
train_labels = strat_train_set['median_house_value'].copy()  #若不加copy()则两变量指向同一个区域，修改一个则另一个也被修改

In [3]:
#统计数据中na的数目，value_counts只支持Series, 发现total_bedrooms中有158个na
# for columns in rep_train.columns:
#     print(columns, rep_train[columns].isna().value_counts())、

#数据清洗
#处理数据缺失的问题：1.去掉对应的街区;2.去掉整个属性;3.进行赋值(0、平均值、中位数等等)
#1.rep_train.dropna(subset=['total_bedrooms'])  #subset需要进行处理的列
#2.rep_train.drop(['total_bedrooms'], axis=1)
#3.median = rep_train['total_bedrooms'].median()
  #rep_train['total_bedrooms'].fillna(median)
#sklearn中提供了方便的类来处理缺失值
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
rep_train_num = rep_train.drop('ocean_proximity', axis=1) # 因为ocean_proximity列不是数据，无法计算中位数，所以去掉
imputer.fit(rep_train_num)
# imputer.statistics_# 查看每列的median。rep_train_num.median().values同样的效果
X = imputer.transform(rep_train_num)# 对缺失数据进行填充,返回值为numpy数组
housing_tr = pd.DataFrame(X, columns=rep_train_num.columns)

#处理文本和类别属性
#sklearn为将文本转换为数据提供了一个转换器

#LabelEncoder适用与Label(只有一列的文本特征)(n_sample,)
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# housing_cat = rep_train['ocean_proximity']
# housing_cat_encoded = encoder.fit_transform(housing_cat)
# encoder.classes_# 查看映射表

#使用LabelBinarizer可以将文本转换为数字，将数字转换为独热码这两步结合

#对于文本属性，使用factorize较优
housing_cat = rep_train['ocean_proximity']
housing_cat_encoded, housing_categories = housing_cat.factorize() # 返回类型编码， 编码所对应的类型集合
#factorize按类型返回的是[0, 1, 2, 3, 4](有5个类别)，这会在训练时使得算法认为0和1比0和5更相似，要解决这个问题，采用独热编码
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(categories='auto')
housing_cat_1hot = one_hot_encoder.fit_transform(housing_cat_encoded.reshape(-1, 1)) # 输出结果是SCIPY稀疏矩阵，存储大量的0浪费空间

In [4]:
# 自定义转换器
'''转换器有一个超参数add_bedrooms_per_room.这个超参数可以让你方便地发现添加了这个属性是否对机器学习算法有帮助。
你可以为每个不能完全确保的数据准备步骤添加一个超参数。数据准备步骤越自动化,可以自动化的操作组合就越多,越容易发现更好用的组合(并能节省大量
时间)。'''
from sklearn.base import TransformerMixin #自定义转换器继承TransformerMixin可以拥有fit_transform功能
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
class CombinedAttributes(TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room;
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
attr_adder = CombinedAttributes(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(df.values)
#特征缩放
'''除了个别情况,当输入的数值属性量度不同时,机器学习算法的性能都不会好。通常情况下我们不需要对目标值进行缩放。有两种常见的方法，
线性函数归一化，即将数据压缩至0-1(MinMaxScaler()函数). 2.标准化0均值单位方差(StandardScaler)。与归一化不同,标准化不会
限定值到某个特定的范围,这对某些算法可能构成问题(比如,神经网络常需要输入值得范围是 0 到 1)。但是,标准化受到异常值的影响很小。
例如,假设一个街区的收入中位数由于某种错误变成了100,归一化会将其它范围是 0 到 15 的值变为 0-0.15,但是标准化不会受什么影响。'''

'除了个别情况,当输入的数值属性量度不同时,机器学习算法的性能都不会好。通常情况下我们不需要对目标值进行缩放。有两种常见的方法，\n线性函数归一化，即将数据压缩至0-1(MinMaxScaler()函数). 2.标准化0均值单位方差(StandardScaler)。与归一化不同,标准化不会\n限定值到某个特定的范围,这对某些算法可能构成问题(比如,神经网络常需要输入值得范围是 0 到 1)。但是,标准化受到异常值的影响很小。\n例如,假设一个街区的收入中位数由于某种错误变成了100,归一化会将其它范围是 0 到 15 的值变为 0-0.15,但是标准化不会受什么影响。'

In [5]:
#转换流水线
#Scikit-Learn 没有工具来处理DataFrame,因此我们需要写一个简单的自定义转换器来做这项工作:
class DataFrameSelector(TransformerMixin):
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name;
    def fit(self, X, y=None):
        return self;
    def transform(self, X, y=None):
        if(X[self.attribute_name].shape[1]==1):
            return X[self.attribute_name].values.reshape(-1,)
        else:
            return X[self.attribute_name].values# .values返回的是numpy.array
'''The pipeline is assuming LabelEncoder's fit_transform method is defined to take three positional arguments:
def fit_transform(self, x, y)
while it is defined to take only two:
def fit_transform(self, x)'''
#因此自己根据LabelEncoder编写自定义转换器
class CharToInt(TransformerMixin):
    def __init__(self):
        self.encoder = LabelEncoder()
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        encoder = LabelEncoder()
        data_encoded = self.encoder.fit_transform(X)
        print(self.encoder.classes_)
        return data_encoded.reshape((-1,1))
        
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder #这里为了展示流水线而将标签编码应用与属性编码(此处只转换一列，故可以应用)
from sklearn.pipeline import FeatureUnion
num_attribts = list(rep_train_num.columns)
cat_attribts = ['ocean_proximity']
#数字数据处理流水线
num_pipeline = Pipeline(
    [('select', DataFrameSelector(num_attribts)),
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributes()),
    ('std_scaler', StandardScaler())
    ])
#文本数据处理流水线
cat_pipeline = Pipeline([
    ('select', DataFrameSelector(cat_attribts)),
    ('char_to_int', CharToInt()),
    ('1hot', OneHotEncoder(categories='auto',sparse=False))
    ])
#将两个流水线并行执行，等待输出，最后将结果合并起来，和np.c_[]合并方式一样
full_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline),
    ])
housing_prepared = full_pipeline.fit_transform(rep_train)

['<1H OCEAN' 'INLAND' 'ISLAND' 'NEAR BAY' 'NEAR OCEAN']


In [6]:
#6
#选择并训练模型
#训练
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, train_labels)

#查看一下前5个数据的预测值
# some_data = housing_prepared[:5]
# some_label = train_labels[:5]
# some_data_predict = lin_reg.predict(some_data)
# print(some_label, '\n', some_data_predict)

#查看训练集的RMSE 
from sklearn.metrics import mean_squared_error
housing_predict = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(train_labels, housing_predict)
lin_rmse = np.sqrt(lin_mse)  # 为68628.19819848923
'''大多数街区的median_housing_values位于 120000到 265000 美元之间,因此预测误差 68628 美元不能让人满意。这是一个模型欠拟合训练数
据的例子。当这种情况发生时,意味着特征没有提供足够多的信息来做出一个好的预测,或者模型并不强大。就像前一章看到的,修复欠拟合的主要方法是
选择一个更强大的模型,给训练算法提供更好的特征,或去掉模型上的限制。这个模型还没有正则化,所以排除了最后一个选项。
你可以尝试添加更多特征(比如,人口的对数值),但是首先让我们尝试一个更为复杂的模型,看看效果。'''
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, train_labels)
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(train_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse) # 为0.0，很有可能是模型发生了过拟合，因此使用交叉验证对模型评估
# 交叉验证
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, train_labels,
                         scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
# print('scores:', rmse_scores, '\n', 'mean:', rmse_scores.mean(), '\n', 'std:', rmse_scores.std())
# 可以看出，模型的性能比线性回归的还要差，可以判断为过拟合

# 随机森林
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=10)
forest_reg.fit(housing_prepared, train_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(train_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse) 


'''不要在调节超参数上花费太多时间。目标是列出一个可能模型的列表(两到五个)。'''

'不要在调节超参数上花费太多时间。目标是列出一个可能模型的列表(两到五个)。'

In [7]:
#7
#模型微调
'''假设你现在有了一个列表,列表里有几个有希望的模型。你现在需要对它们进行微调(超参数)'''
#网格搜索
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(estimator=forest_reg, param_grid=param_grid, cv=5, n_jobs=8)
grid_search.fit(housing_prepared, train_labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=8,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [8]:
#使用网格搜索后的最优模型进行预测
forest_reg = grid_search.best_estimator_
forest_reg.fit(housing_prepared, train_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(train_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse) 
forest_rmse

cvres = grid_search.cv_results_ #显示每一对超参数的得分和参数
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.6855238804209186 {'max_features': 2, 'n_estimators': 3}
0.7663731242472611 {'max_features': 2, 'n_estimators': 10}
0.7904943499115977 {'max_features': 2, 'n_estimators': 30}
0.7254525028665252 {'max_features': 4, 'n_estimators': 3}
0.7904481052115742 {'max_features': 4, 'n_estimators': 10}
0.8079573699843191 {'max_features': 4, 'n_estimators': 30}
0.7467826890484041 {'max_features': 6, 'n_estimators': 3}
0.7977191191513435 {'max_features': 6, 'n_estimators': 10}
0.8129806059675941 {'max_features': 6, 'n_estimators': 30}
0.7459228956279965 {'max_features': 8, 'n_estimators': 3}
0.7989559752725026 {'max_features': 8, 'n_estimators': 10}
0.8131401707454239 {'max_features': 8, 'n_estimators': 30}
0.7065517866344345 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
0.7786930523434362 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
0.7325229767587783 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
0.7948854064312149 {'bootstrap': False, 'max_features': 3, 'n

In [13]:
#随机搜索
'''当超参数的搜索空间很大时,随机搜索RandomizedSearchCV，优点：如果你让随机搜索运行,比如 1000 次,它会探索每个超参数的 1000 个不同的值
(而不是像网格搜索那样,只搜索每个超参数的几个值)。你可以方便地通过设定搜索次数,控制超参数搜索的计算量。'''
#8
#分析最佳模型和他们的误差
#分析最佳模型，指出每个属性对于做出预测值的重要性
feature_import = grid_search.best_estimator_.feature_importances_
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_one_hot_attribs = ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
attributes = num_attribts + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_import, attributes), reverse=True)
'''有了这个信息,你就可以丢弃一些不那么重要的特征(比如,显然只要一个ocean_proximity的类型(INLAND)就够了,所以可以丢弃掉其它的),你还应该
看一下系统犯的误差,搞清为什么会有些误差,以及如何改正问题(添加更多的特征,或相反,去掉没有什么信息的特征,清洗异常值等等)'''

[(0.3780032658537155, 'median_income'),
 (0.16040106764518214, 'INLAND'),
 (0.11268142582772255, 'pop_per_hhold'),
 (0.06740474363119267, 'bedrooms_per_room'),
 (0.06412963967564875, 'longitude'),
 (0.0580162746589019, 'latitude'),
 (0.04339447072841226, 'housing_median_age'),
 (0.041112823097615325, 'rooms_per_hhold'),
 (0.015594366756121589, 'total_rooms'),
 (0.015431603778023571, 'population'),
 (0.015086311677596064, 'total_bedrooms'),
 (0.014671403478780825, 'households'),
 (0.009488025581239911, '<1H OCEAN'),
 (0.0028491515888301533, 'NEAR OCEAN'),
 (0.001643918414461886, 'NEAR BAY'),
 (9.150760655479202e-05, 'ISLAND')]

In [ ]:
# 早停
# from sklearn.base import clone
# if val_error < minimum_val_error:
#     minimum_val_error = val_error
#     best_epoch = epoch
#     best_model = clone(sgd_reg)

In [14]:
#用测试集评估系统
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()
X_test_prepared = full_pipeline.transform(X_test) #这里使用的是transform而不是fit_transform,因为之前fit过了
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
'''在测试集上的性能较差时，一定不要去调节超参数，使得测试集的效果变好，这样的提升不能推广到新数据上'''
# => evaluates to 48,209.6

['<1H OCEAN' 'INLAND' 'ISLAND' 'NEAR BAY' 'NEAR OCEAN']


In [ ]:
import numpy as np
import pandas as pd

#def f(x):
#    return pd.Series([x.max(), x.min()], index=['max', 'min'])
#f = lambda x: x.max() - x.min()
f = lambda x: '%.2f' % x
df = pd.DataFrame(np.arange(12).reshape(3, 4))
print(df)
deal_df = df.applymap(f)
print(deal_df)